In [ ]:
import os
import requests

#params = {"apikey": os.environ["DJANGO_OMDB_KEY"], "t": "starwars"}
params = {"apikey": "", "t": "starwars"}


resp = requests.get("https://www.omdbapi.com/", params=params)
# actual URL will be https://www.omdbapi.com/?apikey=<key>&t=star+wars

print(resp.json())


{'Title': 'Starwars: Goretech', 'Year': '2018', 'Rated': 'N/A', 'Released': '07 Dec 2018', 'Runtime': '90 min', 'Genre': 'Action, Comedy, Sci-Fi', 'Director': 'Germán Magariños', 'Writer': 'Vic Cicuta, Germán Magariños', 'Actors': 'Vic Cicuta, Julieta Grimaldo, Fabian Moreno', 'Plot': 'N/A', 'Language': 'Spanish', 'Country': 'Argentina', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNTI5OTBhMGYtNTZlNS00MjMzLTk5NTEtZDZkODM5YjYzYmE5XkEyXkFqcGdeQXVyMzU0OTU0MzY@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '4.7/10'}], 'Metascore': 'N/A', 'imdbRating': '4.7', 'imdbVotes': '40', 'imdbID': 'tt9336300', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}


In [9]:
resp.json()

{'Title': 'Starwars: Goretech',
 'Year': '2018',
 'Rated': 'N/A',
 'Released': '07 Dec 2018',
 'Runtime': '90 min',
 'Genre': 'Action, Comedy, Sci-Fi',
 'Director': 'Germán Magariños',
 'Writer': 'Vic Cicuta, Germán Magariños',
 'Actors': 'Vic Cicuta, Julieta Grimaldo, Fabian Moreno',
 'Plot': 'N/A',
 'Language': 'Spanish',
 'Country': 'Argentina',
 'Awards': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNTI5OTBhMGYtNTZlNS00MjMzLTk5NTEtZDZkODM5YjYzYmE5XkEyXkFqcGdeQXVyMzU0OTU0MzY@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '4.7/10'}],
 'Metascore': 'N/A',
 'imdbRating': '4.7',
 'imdbVotes': '40',
 'imdbID': 'tt9336300',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',
 'Website': 'N/A',
 'Response': 'True'}

In [7]:
#params = {"apikey": os.environ["DJANGO_OMDB_KEY"], "s": "starwar", "type": "movie"}
params = {"apikey": "81d0a701", "s": "starwar", "type": "movie"}

resp = requests.get("https://www.omdbapi.com/", params=params)
resp.json()

{'Response': 'False', 'Error': 'Movie not found!'}

In [3]:
import logging
import requests

logger = logging.getLogger(__name__)
OMDB_API_URL = "https://www.omdbapi.com/"



class OmdbMovie:
    """A simple class to represent movie data coming back fromOMDb
    and transform to Python types.
    """
    def __init__(self, data):
        """Data is the raw JSON/dict returned from OMDb"""
        self.data = data

    def check_for_detail_data_key(self, key):
        """Some keys are only in the detail response, raise anexception if the key is not found."""
        if key not in self.data:
            raise AttributeError(f"{key} is not in data, please make sure this is a detail response.")
        
    @property
    def imdb_id(self):
        return self.data["imdbID"]
    
    @property
    def title(self):
        return self.data["Title"]
    
    @property
    def year(self):
        return int(self.data["Year"])
    
    @property
    def runtime_minutes(self):
        self.check_for_detail_data_key("Runtime")
        rt, units = self.data["Runtime"].split(" ")
        if units != "min":
            raise ValueError(f"Expected units 'min' for runtime. Got '{units}")
        
        return int(rt)


    @property
    def genres(self):
        self.check_for_detail_data_key("Genre")
        return self.data["Genre"].split(", ")
    
    @property
    def plot(self):
        self.check_for_detail_data_key("Plot")
        return self.data["Plot"]


In [2]:
# data is a dictionary from the API
data = {
    "Title": "My Great Movie", 
    "Year": "1991"
}

movie = OmdbMovie(data)

In [4]:
movie.title

'My Great Movie'

In [5]:
movie.year

1991

In [7]:
#movie.plot

In [8]:
class OmdbClient:
    def __init__(self, api_key):
        self.api_key = api_key

def make_request(self, params):
    """Make a GET request to the API, automatically adding
    the `apikey` to parameters."""
    params["apikey"] = self.api_key
    resp = requests.get(OMDB_API_URL, params=params)
    resp.raise_for_status()
    return resp

def get_by_imdb_id(self, imdb_id):
    """Get a movie by its IMDB ID"""
    logger.info("Fetching detail for IMDB ID %s", imdb_id)
    resp = self.make_request({"i": imdb_id})
    return OmdbMovie(resp.json())

def search(self, search):
    """Search for movies by title. This is a generator so
    all results from all pages will be iterated across."""
    page = 1
    seen_results = 0
    total_results = None
    logger.info("Performing a search for '%s'", search)

    while True:
        logger.info("Fetching page %d", page)
        resp = self.make_request({"s": search, "type":
        "movie", "page": str(page)})
        resp_body = resp.json()

        if total_results is None:
            total_results = int(resp_body["totalResults"])
    
        for movie in resp_body["Search"]:
            seen_results += 1
            yield OmdbMovie(movie)

        if seen_results >= total_results:
            break
        
        page += 1

In [9]:
#from django.conf import settings
# from omdb.client import OmdbClient

def get_client_from_settings():
    """Create an instance of an OmdbClient using the OMDB_KEY."""
    OMDB_KEY = "81d0a701"
    
    return OmdbClient(OMDB_KEY)


## Helper Functions

Rather than integrate OmdbClient directly into our Django views, we’ll write three helper functions that contain the logic. This will allow to add some
management commands without repeating code. The helper functions will be:

- `get_or_create_genres()`: Accepts a list of genre names (strings) and yields a Genre object for each of them.
- ``fill_movie_details()``: Accepts a Movie object and then queries the OMDb API to fill in the missing data.
- ``search_and_save()``: Accepts a search term (string). Checks if the search has been performed in the past 24 hours, and returns if so. Otherwise, it performs the search against the OMDb API and creates Movie objects for each result.


In [ ]:
import logging
import re
from datetime import timedelta
from django.utils.timezone import now
from movies.models import Genre, SearchTerm, Movie
from omdb.django_client import get_client_from_settings
logger = logging.getLogger(__name__)


def get_or_create_genres(genre_names):
    for genre_name in genre_names:
        genre, created = Genre.objects.get_or_create(name=genre_name)
    yield genre


def fill_movie_details(movie):
    """
    Fetch a movie's full details from OMDb. Then, save it to the
    DB. If the movie already has a `full_record` this does
    nothing, so it's safe to call with any `Movie`.
    """
    if movie.is_full_record:
        logger.warning("'%s' is already a full record.", movie.title,)
    return

    omdb_client = get_client_from_settings()
    movie_details = omdb_client.get_by_imdb_id(movie.imdb_id)
    movie.title = movie_details.title
    movie.year = movie_details.year
    movie.plot = movie_details.plot
    movie.runtime_minutes = movie_details.runtime_minutes
    movie.genres.clear()

    for genre in get_or_create_genres(movie_details.genres):
        movie.genres.add(genre)
    movie.is_full_record = True
    movie.save()



def search_and_save(search):
    """
    Perform a search for search_term against the API, but only
    if it hasn't been searched in the past 24 hours. Save
    each result to the local DB as a partial record.
    """
    # Replace multiple spaces with single spaces, and lowercase the search
    normalized_search_term = re.sub(r"\s+", " ", search.lower())
    search_term, created = SearchTerm.objects.get_or_create(term=normalized_search_term)

    if not created and (search_term.last_search > now() - timedelta(days=1)):
        # Don't search as it has been searched recently
        logger.warning("Search for '%s' was performed in the past 24 hours so not searching again.",
                        normalized_search_term,)
        return

    omdb_client = get_client_from_settings()

    for omdb_movie in omdb_client.search(search):
        logger.info("Saving movie: '%s' / '%s'",
        omdb_movie.title, omdb_movie.imdb_id)
        movie, created = Movie.objects.get_or_create(
            imdb_id=omdb_movie.imdb_id,
            defaults={
                "title": omdb_movie.title,
                "year": omdb_movie.year,
            },
        )

        if created:
            logger.info("Movie created: '%s'", movie.title)

    search_term.save()

In [ ]:
python3 movie-night/module3/movienight/manage.py runserver 0.0.0.0:8000


python3 movie-night/module3/movienight/manage.py createsuperuser